In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import sklearn.metrics as sm 
import time
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import RobustScaler
import pyarrow.parquet as pq
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

#pip install feature-engine to install the windorizor feature
#!pip install 

### Reading from the modified data after applying the groupby and other operations

### The pre processed data was converted to a new parquet file for easy loading usage
>> the link for feature engineered data can be accessed through stevens id

>> https://stevens0-my.sharepoint.com/:u:/g/personal/aagarw16_stevens_edu/EfujhQL1KL1CvufiGin3iUMB7AUW3NMutstgDMe1muEwsw?e=BaaD2f

In [3]:
df= pd.read_parquet('C:/Users/aagarw16/OneDrive - stevens.edu/Desktop/ML Project/Project/Data_transformed.parquet')

In [4]:
df.drop("Unnamed: 0",axis=1,inplace=True)

### Since after pre-processing there is no use of the Customer ID so that column was made in to the index

In [5]:
df.set_index("customer_ID")

P_2_mean   P_2_std   P_2_min   P_2_max  P_2_last  \
customer_ID                                                              
-9223358381327749917  0.415868  0.057145  0.340178  0.498727  0.387708   
-9223193039457028513  0.974068  0.013094  0.964483  1.002477  1.001372   
-9223189665817919541  0.802447  0.038025  0.694073  0.828761  0.694073   
-9223188534444851899  0.791203  0.002688  0.786647  0.794826  0.787945   
-9223173911659837606  0.115666  0.078554  0.038207  0.252421  0.040486   
...                        ...       ...       ...       ...       ...   
 9223275399009481284  0.508722  0.034856  0.446763  0.580708  0.502093   
 9223300670094807586  0.128279  0.138331 -0.035435  0.395510  0.118304   
 9223303087902649707  0.364807  0.074824  0.271125  0.517709  0.368901   
 9223345210145379887  0.848886  0.059538  0.776463  0.940111  0.902587   
 9223350112805974911  0.232830  0.064645  0.102516  0.305949  0.305828   

                      D_39_mean   D_39_std  D_39_min  D_39_max  D_39_last  \
customer_ID                                                                 
-9223358381327749917   2.615385   4.628507         0        16          0   
-9223193039457028513   0.000000   0.000000         0         0          0   
-9223189665817919541   0.000000   0.000000         0         0          0   
-9223188534444851899   0.000000   0.000000         0         0          0   
-9223173911659837606   4.384615   6.144625         0        17         13   
...                         ...        ...       ...       ...        ...   
 9223275399009481284   3.769231   8.427671         0        31          8   
 9223300670094807586   5.461538   6.213096         1        22          6   
 9223303087902649707   3.692308   8.576952         0        23          0   
 9223345210145379887   4.000000   7.118052         0        23          0   
 9223350112805974911   8.538462  11.730412         0        33         31   

                      ...  D_64_count  D_64_last  D_64_nunique  D_66_count  \
customer_ID           ...                                                    
-9223358381327749917  ...          13          2             1          13   
-9223193039457028513  ...          13          0             1          13   
-9223189665817919541  ...          13          0             1          13   
-9223188534444851899  ...          13          3             2          13   
-9223173911659837606  ...          13          0             2          13   
...                   ...         ...        ...           ...         ...   
 9223275399009481284  ...          13          3             1          13   
 9223300670094807586  ...          13          3             2          13   
 9223303087902649707  ...          13          2             1          13   
 9223345210145379887  ...          13          2             1          13   
 9223350112805974911  ...          13          0             2          13   

                      D_66_last  D_66_nunique  D_68_count  D_68_last  \
customer_ID                                                            
-9223358381327749917         -1             1          13          3   
-9223193039457028513         -1             1          13          6   
-9223189665817919541         -1             1          13          6   
-9223188534444851899         -1             1          13          5   
-9223173911659837606         -1             1          13          6   
...                         ...           ...         ...        ...   
 9223275399009481284         -1             1          13          6   
 9223300670094807586         -1             1          13          5   
 9223303087902649707         -1             1          13          3   
 9223345210145379887         -1             1          13          6   
 9223350112805974911         -1             1          13          6   

                      D_68_nunique  target  
customer_ID                                 
-92233583813277

In [6]:
X = df.drop(["target","customer_ID"],axis=1)
y = df['target']

y_value=((X < (Q1_1 - 1.5 * IQR_1)) | (X > (Q3_1 + 1.5 * IQR_1))).sum()
y_value_1 = (((y_value/X.shape[0])*100)>15)

# TRIED MORE METHOD FOR OUTLIER TREATMENT BUT NONE WAS WORKING CORRECTLY FOR OUR DATA SET

y_value_2 = list(y_value_1[y_value_1==True].index)
y_value_2[:15]

windsorizer = Winsorizer(capping_method='iqr', tail='both',fold=1.5,variables=y_value_2[:15])
X_new = windsorizer.fit_transform(X)

X_median = X.median()

X = X.mask(((X < (Q1_1 - 1.5 * IQR_1)) | (X > (Q3_1 + 1.5 * IQR_1))), X.median(),axis=1)

X.isna().sum().sum()

# SPITTING DATA SET INTO TEST AND TRAIN

Q1_1 = X.quantile(0.25)
Q3_1 = X.quantile(0.75)
IQR_1 = Q3_1 - Q1_1
y_value=((X < (Q1_1 - 1.5 * IQR_1)) | (X > (Q3_1 + 1.5 * IQR_1))).sum()
(((y_value/X.shape[0])*100)>15).value_counts()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=30)

In [ ]:
# XGBoost model was first trained for the pre-preocessed data set with the default parameters

In [13]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
print(sm.f1_score(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93    101020
           1       0.80      0.80      0.80     35118

    accuracy                           0.90    136138
   macro avg       0.87      0.86      0.86    136138
weighted avg       0.90      0.90      0.90    136138

0.7989158345221113


# These parameters are the most optimum parameter that we found after running a Grid Search CV with multiple parameters values. The grid search code os given below

In [79]:
params = {
            'objective': 'binary:logistic', 
            'max_depth': 8,
            'subsample':0.88,
            'colsample_bytree': 0.5,
            'gamma':1.5,
            'min_child_weight':8,
            'lambda':50,
            'eta':0.03,
    }
model = xgb.XGBClassifier(**params)


In [80]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False, eta=0.03,
              eval_metric=None, feature_types=None, gamma=1.5, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', lambda=50, learning_rate=0.0299999993,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=8,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, ...)

In [33]:
y_pred = model.predict(X_test)

In [82]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
print(sm.f1_score(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93    101020
           1       0.80      0.80      0.80     35118

    accuracy                           0.90    136138
   macro avg       0.87      0.87      0.87    136138
weighted avg       0.90      0.90      0.90    136138

0.8020268341421639


# We also ran the StratifiedkFold to find out what is the best value of accuracies we can get for out data set with different train and validation set 

In [ ]:
kfold = StratifiedKFold(n_splits=5)
accuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = kfold)

In [40]:
accuracies

array([0.89583202, 0.89652459, 0.89630422, 0.89535982, 0.89819304,
       0.8974028 , 0.89636392, 0.89608059, 0.89762317, 0.89768613])

# We also ran the StratifiedkFold to find out what is the best value of f1 score we can get for out data set with different train and validation set 

In [ ]:
model_CV = xgb.XGBClassifier(**params)
kfold = StratifiedKFold(n_splits=5)
results = cross_val_score(model_CV, X, y, cv=kfold,scoring='f1')
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [43]:
results

array([0.79751366, 0.79777588, 0.80288379, 0.80104142, 0.80020617])

# Parameter list for grid search with fixed values Binaary: logistic is bused as it is a binarry classification

In [68]:
estimator = xgb.XGBClassifier(objective= 'binary:logistic', 
            subsample=0.88,
            colsample_bytree= 0.5,
            gamma=1.5,
            min_child_weight=8
            
)

# Looping through different values of other parameters 

In [72]:
params= {'eta':[0.03,0.01],
'max_depth':[7,8],'lambda':[70,50]}

In [ ]:
# Executing Grid Search CV too estimate the best parameter list

In [75]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=params,
    scoring = 'f1',
    cv = 5,
    verbose=True)

In [76]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.5,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=1.5, gpu_id=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=8,
     

# finding out the best parameter values after grid search CV

In [78]:
grid_search.best_estimator_


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False, eta=0.03,
              eval_metric=None, feature_types=None, gamma=1.5, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', lambda=50, learning_rate=0.0299999993,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=8,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, ...)

# Using the hyper parameter tuned XG boost algorithm, we got a accuracy which improved by 0.02% this improvement might not look much but considering the data set size, almost 2000 data point where then correctly classified. With more time and better outlier treatment and behaviour of data set understanding, The accuracy can be improved further. 